### Product And Location

In [1]:
# pip install geopy

In [2]:
import unicodedata
import datetime as dt
import os
import sys
import re
import numpy as np
import pandas as pd
import math
from pathlib import Path
import nltk
from nltk import word_tokenize 
import shutil
from os.path import isfile, join
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import locale
from locale import atof
from geopy.geocoders import Nominatim
from geopy.point import Point
from geopy.exc import GeocoderTimedOut
from functools import reduce

##### Functions

In [3]:
# custom character alphabet for word
tr = re.compile(r"[abcçdefgğhıijklmnoöprsştuüvyzqxw]+", re.IGNORECASE|re.UNICODE) # Turkish filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n0123456789'
en = re.compile(r"[abcdefghıijklmnopqrstxuvwyz]+", re.IGNORECASE|re.UNICODE) # English
nl = re.compile(r"[abcdefghıijklmnopqrstxuvwyzāăēĕīĭōŏūŭ]+", re.IGNORECASE|re.UNICODE) # Dutch (Flemenk)
fr = re.compile(r"[abcçdefghıijklmnopqrstxuvwyzàâæèéêëîïôœùûüÿ]+", re.IGNORECASE|re.UNICODE) # French
de = re.compile(r"[abcdefghıijklmnopqrstxuvwyzäöüß]+", re.IGNORECASE|re.UNICODE) # German
es = re.compile(r"[abcdefghıijklmnopqrstxuvwyzñáéíóú]+", re.IGNORECASE|re.UNICODE) # Spanish (¿¡)
pt = re.compile(r"[abcçdefghıijklmnopqrstxuvwyzàáâãéêíóôõú]+", re.IGNORECASE|re.UNICODE) # Portuguese
it = re.compile(r"[abcdefghıijklmnopqrstxuvwyzàéèìòùî]+", re.IGNORECASE|re.UNICODE) # Italian
ar = re.compile(r"[ٿصؼۤڳڲؿڎػڠجڿ٬ٸؽؒؓطۄڀۂؘؔتٚڛےٝڜؖڦ٫ډ۰زۇٖۀ،لۓعٮێڔ۶ؚۧۜڤۏإٞٷؗۖ؈ژۣؕؑٴأۻڸۺگاڴڹۯ؉ْڌ؍ي؟ـٟړۅؐڶُىڽېًۢؠضۚڄٛڏٱۦ٩س٦ڼڂٔۘ٠ښٌٍ۬ٳ۾ٲږذۋٵٜ٘ڞڅںٗهڣۿپڒۥۗڋیؙم؞ثۨٹڵڪظٶۭ١ڭەڨحٕ؎ٺڷٰ۪۫ڻڥۛڑڟټآڡغګ؊ّٯڧڮ؏ۮ؋ؤ٪ؾڗۼق۟دکوِڰڐۃ۽ہفرڇچڝ۴بۈٽڕۡھةٓڃئ؛ڬٙڙڢڱۊَۆۉځ۠ۍۑۙڊنءڈٻشڍ؇۵كخ\ا]+", re.IGNORECASE|re.UNICODE) # Arabic

In [4]:
def convert_one_character_letter(text):
    '''This function converts two byte occupy of letter to one byte unicode character without any visual change \n
    like as Turkish character ç,ş,ö,ğ. \n
    convert_one_character_letter(text): text is any string word or sentence.
    '''
    new_text = unicodedata.normalize('NFC', f"{text}")
    return new_text

In [5]:
def clean_text(text, custom_alp=tr): # for only string in specific language
    '''This function extract custom_alp character from string \n
    clean_text(text, custom_alp): text is any string word or sentence and custom_alp is specific alphabet.
    '''
    # text_clean = re.findall(custom_alp, text)
    text_result_list = []
    text_list = re.findall(r'\S+', text)
    for text_var in text_list:
        text_clean = re.findall(custom_alp, str(text_var))
        text_result_var = "".join(text_clean)
        text_result_list.append(text_result_var)
    text_result = " ".join(text_result_list)
    return text_result

In [6]:
def lower_func(text):
    '''This function convert string character to lowercase. \n
    lower_func(text=str): text is any string word or sentence
    '''
    string_lower = str.lower(text)
    return string_lower

In [7]:
def upper_func(text):
    '''This function convert string character to uppercase. \n
    upper_func(text=str): text is any string word or sentence
    '''
    string_upper = str.upper(text)
    return string_upper

In [8]:
def capitalize_func(text):
    '''This function convert string character to capitalize case. \n
    capitalize_func(text=str): text is any string word or sentence
    '''
    string_capitalize = str.capitalize(text)
    return string_capitalize

In [9]:
def title_func(text):
    '''This function convert string character to title as each word start with capital letter. \n
    title_func(text=str): text is any string word or sentence
    '''
    string_title = str.title(text)
    return string_title

In [10]:
def character_clean_lower_text(text, custom_alp=tr): # for only string in specific language
    '''This function converts two byte occupy of letter to one byte unicode character without any visual change \n
    like as Turkish character ç,ş,ö,ğ. And also extract custom_alp character from string and convert to lowercase. \n
    character_clean_lower_text(text, custom_alp): text is any string word or sentence and custom_alp is specific alphabet.  
    '''
    new_text = unicodedata.normalize('NFC', f"{text}")
    text_result_list = []
    text_list = re.findall(r'\S+', new_text)  # like as word tokenize
    for text_var in text_list:
        text_clean = re.findall(custom_alp, str(text_var))
        text_result_var = "".join(text_clean)
        text_result_list.append(text_result_var)
    text_join = " ".join(text_result_list)
    text_result = str.lower(text_join)
    return text_result

In [11]:
def convert_to_float(value):
    '''This function convert string numeric value to float type. Like as 1.200,15
    convert_to_float(value): value is a string numeric value
    '''
    value = value.replace(".","").replace(",",".")
    return float(value)

In [12]:
def whitespace_del(text):
    '''This function provides removing left right white space.\n
    whitespace_del(text): text is a string value as word or sentence.
    '''
    text_var = str(text)
    text_var = text_var.strip()  # rstrip lstrip
    return text_var

#### Read Data Analysis

In [13]:
variable_date = "01.04.2024-05.09.2024"
input_path = fr"C:\Users\user\Desktop\Data Analysis\Adoba\Sales_Report\Data\Entegra\Sales\{variable_date}"
output_path = fr"C:\Users\user\Downloads"

data_file = fr"Entegra_Sales_List ({variable_date})"

In [14]:
#df_entegra = pd.read_csv(fr"{input_path}\{data_file}.csv", low_memory=False)
df_entegra = pd.read_excel(fr"{input_path}\{data_file}.xlsx")
df_entegra

,id,order_number,platform_reference_no,invoice_number,datetime,date_add,entegration,supplier,status,status_name,...,pov_productCode,payment_type,total_product,PazaryerindenGelenOdemeTutar,total_product_quantity,delivery_method,fatura_tarihi,invoice_url,invoice_type,FaturaTipiAdi
0,201036,2383110310,9465795799,NaN,2024-09-05 10:54:27,2024-09-05 10:57:15,trendyol,trendyol,2,Onaylandı,...,NaN,Trendyol,1,0,1,Aynı gün kargoda,NaN,NaN,0,Seciniz
1,201035,2383102583,9465789564,NaN,2024-09-05 10:50:42,2024-09-05 10:51:14,trendyol,trendyol,2,Onaylandı,...,NaN,Trendyol,5,0,7,Aynı gün kargoda,NaN,NaN,0,Seciniz
2,201033,2383091113,9465777317,NaN,2024-09-05 10:43:15,2024-09-05 10:45:03,trendyol,trendyol,2,Onaylandı,...,NaN,Trendyol,1,0,1,Aynı gün kargoda,NaN,NaN,0,Seciniz
3,201025,2383085464,9465772854,NaN,2024-09-05 10:40:26,2024-09-05 10:42:12,trendyol,trendyol,2,Onaylandı,...,NaN,Trendyol,2,0,2,Aynı gün kargoda,NaN,NaN,0,Seciniz
4,201027,2383076067,9465764076,NaN,2024-09-05 10:34:55,2024-09-05 10:42:15,trendyol,trendyol,2,Onaylandı,...,220647971025,Trendyol,1,0,1,Aynı gün kargoda,NaN,NaN,0,Seciniz
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92975,107816,2036971893,9080057915,NaN,2024-03-01 00:16:54,2024-03-01 00:17:34,trendyol,trendyol,2,Onaylandı,...,NaN,Trendyol,1,0,1,Ertesi Gün Kargoda,03/01/2024 10:59:38,NaN,0,Seciniz
92976,107817,2036970386,9080056913,NaN,2024-03-01 00:16:07,2024-03-01 00:17:35,trendyol,trendyol,2,Onaylandı,...,NaN,Trendyol,1,0,1,Ertesi Gün Kargoda,03/01/2024 09:21:47,NaN,0,Seciniz
92977,107815,0888556689,0888556689,NaN,2024-03-01 00:14:02,2024-03-01 00:15:22,HepsiBurada,hb,2,Onaylandı,...,NaN,HepsiBurada,1,0,1,NaN,03/01/2024 09:53:35,NaN,0,Seciniz
92978,107812,2036956451,9080042398,NaN,2024-03-01 00:05:22,2024-03-01 00:06:28,trendyol,trendyol,2,Onaylandı,...,yatak54896513739,Trendyol,1,0,1,NaN,03/01/2024 10:59:38,NaN,0,Seciniz


In [15]:
df_entegra.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92980 entries, 0 to 92979
Data columns (total 59 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   id                            92980 non-null  int64         
 1   order_number                  92980 non-null  object        
 2   platform_reference_no         92980 non-null  object        
 3   invoice_number                0 non-null      float64       
 4   datetime                      92980 non-null  datetime64[ns]
 5   date_add                      92980 non-null  datetime64[ns]
 6   entegration                   92980 non-null  object        
 7   supplier                      92980 non-null  object        
 8   status                        92980 non-null  int64         
 9   status_name                   92980 non-null  object        
 10  company                       92979 non-null  object        
 11  cargo_code                  

In [16]:
df_entegra.columns = [x.lower() for x in df_entegra.columns]
df_entegra.columns = [x.strip() for x in df_entegra.columns]
df_entegra

,id,order_number,platform_reference_no,invoice_number,datetime,date_add,entegration,supplier,status,status_name,...,pov_productcode,payment_type,total_product,pazaryerindengelenodemetutar,total_product_quantity,delivery_method,fatura_tarihi,invoice_url,invoice_type,faturatipiadi
0,201036,2383110310,9465795799,NaN,2024-09-05 10:54:27,2024-09-05 10:57:15,trendyol,trendyol,2,Onaylandı,...,NaN,Trendyol,1,0,1,Aynı gün kargoda,NaN,NaN,0,Seciniz
1,201035,2383102583,9465789564,NaN,2024-09-05 10:50:42,2024-09-05 10:51:14,trendyol,trendyol,2,Onaylandı,...,NaN,Trendyol,5,0,7,Aynı gün kargoda,NaN,NaN,0,Seciniz
2,201033,2383091113,9465777317,NaN,2024-09-05 10:43:15,2024-09-05 10:45:03,trendyol,trendyol,2,Onaylandı,...,NaN,Trendyol,1,0,1,Aynı gün kargoda,NaN,NaN,0,Seciniz
3,201025,2383085464,9465772854,NaN,2024-09-05 10:40:26,2024-09-05 10:42:12,trendyol,trendyol,2,Onaylandı,...,NaN,Trendyol,2,0,2,Aynı gün kargoda,NaN,NaN,0,Seciniz
4,201027,2383076067,9465764076,NaN,2024-09-05 10:34:55,2024-09-05 10:42:15,trendyol,trendyol,2,Onaylandı,...,220647971025,Trendyol,1,0,1,Aynı gün kargoda,NaN,NaN,0,Seciniz
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92975,107816,2036971893,9080057915,NaN,2024-03-01 00:16:54,2024-03-01 00:17:34,trendyol,trendyol,2,Onaylandı,...,NaN,Trendyol,1,0,1,Ertesi Gün Kargoda,03/01/2024 10:59:38,NaN,0,Seciniz
92976,107817,2036970386,9080056913,NaN,2024-03-01 00:16:07,2024-03-01 00:17:35,trendyol,trendyol,2,Onaylandı,...,NaN,Trendyol,1,0,1,Ertesi Gün Kargoda,03/01/2024 09:21:47,NaN,0,Seciniz
92977,107815,0888556689,0888556689,NaN,2024-03-01 00:14:02,2024-03-01 00:15:22,HepsiBurada,hb,2,Onaylandı,...,NaN,HepsiBurada,1,0,1,NaN,03/01/2024 09:53:35,NaN,0,Seciniz
92978,107812,2036956451,9080042398,NaN,2024-03-01 00:05:22,2024-03-01 00:06:28,trendyol,trendyol,2,Onaylandı,...,yatak54896513739,Trendyol,1,0,1,NaN,03/01/2024 10:59:38,NaN,0,Seciniz


In [17]:
df_entegra_select_var = df_entegra[["order_number","model","pov_productcode","product_name","datetime","total_product_quantity","entegration",\
    "invoice_city","invoice_country","total","tax","grand_total","company","cargo_company","status_name"]]
df_entegra_select_var

,order_number,model,pov_productcode,product_name,datetime,total_product_quantity,entegration,invoice_city,invoice_country,total,tax,grand_total,company,cargo_company,status_name
0,2383110310,003362,NaN,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-09-05 10:54:27,1,trendyol,İstanbul,NaN,391.670,78.330,470.00,ali özdemir,trendyolekspress,Onaylandı
1,2383102583,004382,NaN,Çok Amaçlı Multi İnce Temizlik Fırçası Derz Ar...,2024-09-05 10:50:42,7,trendyol,İstanbul,NaN,395.530,48.470,444.00,Tuğba Sayar,trendyolekspress,Onaylandı
2,2383091113,001602,NaN,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-09-05 10:43:15,1,trendyol,Ankara,NaN,165.830,33.170,199.00,Tuba Koca,trendyolekspress,Onaylandı
3,2383085464,001602,NaN,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-09-05 10:40:26,2,trendyol,İstanbul,NaN,215.830,43.170,259.00,Doğa Özkan,trendyolekspress,Onaylandı
4,2383076067,002583,220647971025,"Portatif Katlanabilir Masa, Piknik Masası, Kam...",2024-09-05 10:34:55,1,trendyol,Sivas,NaN,181.820,18.180,200.00,Damla Beşer,trendyolekspress,Onaylandı
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92975,2036971893,001603,NaN,"Pikas Ahşap Özel İsim Baskılı, Kare Kedi Köpek...",2024-03-01 00:16:54,1,trendyol,Samsun,NaN,133.330,26.670,160.00,Gülizar Sarı,trendyolekspress,Onaylandı
92976,2036970386,002429,NaN,BTS3 Boyama Tuval Sayılı Set 25x35 Yunus,2024-03-01 00:16:07,1,trendyol,Çorum,NaN,154.170,30.830,185.00,zübeyde DEMİREL,trendyolekspress,Onaylandı
92977,0888556689,002369,NaN,Küçük Çelik Uçlu Kedi Köpek Evcil Hayvan Tırna...,2024-03-01 00:14:02,1,HepsiBurada,İzmir,NaN,33.325,6.665,39.99,İrem Erbey,hepsijet,Onaylandı
92978,2036956451,002284,yatak54896513739,Kalorifer Petek Radyatör Yatağı Kedi Geniş Ked...,2024-03-01 00:05:22,1,trendyol,Konya,NaN,142.500,28.500,171.00,Esma Söylemez,trendyolekspress,Onaylandı


In [18]:
df_entegra_select_var.loc[:,"cargo_company"] = df_entegra_select_var.loc[:,"cargo_company"].fillna("diger")
df_entegra_select_var

,order_number,model,pov_productcode,product_name,datetime,total_product_quantity,entegration,invoice_city,invoice_country,total,tax,grand_total,company,cargo_company,status_name
0,2383110310,003362,NaN,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-09-05 10:54:27,1,trendyol,İstanbul,NaN,391.670,78.330,470.00,ali özdemir,trendyolekspress,Onaylandı
1,2383102583,004382,NaN,Çok Amaçlı Multi İnce Temizlik Fırçası Derz Ar...,2024-09-05 10:50:42,7,trendyol,İstanbul,NaN,395.530,48.470,444.00,Tuğba Sayar,trendyolekspress,Onaylandı
2,2383091113,001602,NaN,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-09-05 10:43:15,1,trendyol,Ankara,NaN,165.830,33.170,199.00,Tuba Koca,trendyolekspress,Onaylandı
3,2383085464,001602,NaN,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-09-05 10:40:26,2,trendyol,İstanbul,NaN,215.830,43.170,259.00,Doğa Özkan,trendyolekspress,Onaylandı
4,2383076067,002583,220647971025,"Portatif Katlanabilir Masa, Piknik Masası, Kam...",2024-09-05 10:34:55,1,trendyol,Sivas,NaN,181.820,18.180,200.00,Damla Beşer,trendyolekspress,Onaylandı
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92975,2036971893,001603,NaN,"Pikas Ahşap Özel İsim Baskılı, Kare Kedi Köpek...",2024-03-01 00:16:54,1,trendyol,Samsun,NaN,133.330,26.670,160.00,Gülizar Sarı,trendyolekspress,Onaylandı
92976,2036970386,002429,NaN,BTS3 Boyama Tuval Sayılı Set 25x35 Yunus,2024-03-01 00:16:07,1,trendyol,Çorum,NaN,154.170,30.830,185.00,zübeyde DEMİREL,trendyolekspress,Onaylandı
92977,0888556689,002369,NaN,Küçük Çelik Uçlu Kedi Köpek Evcil Hayvan Tırna...,2024-03-01 00:14:02,1,HepsiBurada,İzmir,NaN,33.325,6.665,39.99,İrem Erbey,hepsijet,Onaylandı
92978,2036956451,002284,yatak54896513739,Kalorifer Petek Radyatör Yatağı Kedi Geniş Ked...,2024-03-01 00:05:22,1,trendyol,Konya,NaN,142.500,28.500,171.00,Esma Söylemez,trendyolekspress,Onaylandı


In [19]:
df_entegra_select_var.loc[:,"cargo_company"] = df_entegra_select_var.loc[:,"cargo_company"].fillna("diger")
df_entegra_select_var

,order_number,model,pov_productcode,product_name,datetime,total_product_quantity,entegration,invoice_city,invoice_country,total,tax,grand_total,company,cargo_company,status_name
0,2383110310,003362,NaN,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-09-05 10:54:27,1,trendyol,İstanbul,NaN,391.670,78.330,470.00,ali özdemir,trendyolekspress,Onaylandı
1,2383102583,004382,NaN,Çok Amaçlı Multi İnce Temizlik Fırçası Derz Ar...,2024-09-05 10:50:42,7,trendyol,İstanbul,NaN,395.530,48.470,444.00,Tuğba Sayar,trendyolekspress,Onaylandı
2,2383091113,001602,NaN,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-09-05 10:43:15,1,trendyol,Ankara,NaN,165.830,33.170,199.00,Tuba Koca,trendyolekspress,Onaylandı
3,2383085464,001602,NaN,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-09-05 10:40:26,2,trendyol,İstanbul,NaN,215.830,43.170,259.00,Doğa Özkan,trendyolekspress,Onaylandı
4,2383076067,002583,220647971025,"Portatif Katlanabilir Masa, Piknik Masası, Kam...",2024-09-05 10:34:55,1,trendyol,Sivas,NaN,181.820,18.180,200.00,Damla Beşer,trendyolekspress,Onaylandı
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92975,2036971893,001603,NaN,"Pikas Ahşap Özel İsim Baskılı, Kare Kedi Köpek...",2024-03-01 00:16:54,1,trendyol,Samsun,NaN,133.330,26.670,160.00,Gülizar Sarı,trendyolekspress,Onaylandı
92976,2036970386,002429,NaN,BTS3 Boyama Tuval Sayılı Set 25x35 Yunus,2024-03-01 00:16:07,1,trendyol,Çorum,NaN,154.170,30.830,185.00,zübeyde DEMİREL,trendyolekspress,Onaylandı
92977,0888556689,002369,NaN,Küçük Çelik Uçlu Kedi Köpek Evcil Hayvan Tırna...,2024-03-01 00:14:02,1,HepsiBurada,İzmir,NaN,33.325,6.665,39.99,İrem Erbey,hepsijet,Onaylandı
92978,2036956451,002284,yatak54896513739,Kalorifer Petek Radyatör Yatağı Kedi Geniş Ked...,2024-03-01 00:05:22,1,trendyol,Konya,NaN,142.500,28.500,171.00,Esma Söylemez,trendyolekspress,Onaylandı


In [20]:
df_entegra_select_var = df_entegra_select_var.sort_values(["datetime","total_product_quantity"], ascending=False)
df_entegra_select_var.reset_index(drop=True, inplace=True)
df_entegra_select_var

,order_number,model,pov_productcode,product_name,datetime,total_product_quantity,entegration,invoice_city,invoice_country,total,tax,grand_total,company,cargo_company,status_name
0,2383110310,003362,NaN,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-09-05 10:54:27,1,trendyol,İstanbul,NaN,391.670,78.330,470.00,ali özdemir,trendyolekspress,Onaylandı
1,2383102583,004382,NaN,Çok Amaçlı Multi İnce Temizlik Fırçası Derz Ar...,2024-09-05 10:50:42,7,trendyol,İstanbul,NaN,395.530,48.470,444.00,Tuğba Sayar,trendyolekspress,Onaylandı
2,2383091113,001602,NaN,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-09-05 10:43:15,1,trendyol,Ankara,NaN,165.830,33.170,199.00,Tuba Koca,trendyolekspress,Onaylandı
3,2383085464,001602,NaN,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-09-05 10:40:26,2,trendyol,İstanbul,NaN,215.830,43.170,259.00,Doğa Özkan,trendyolekspress,Onaylandı
4,2383076067,002583,220647971025,"Portatif Katlanabilir Masa, Piknik Masası, Kam...",2024-09-05 10:34:55,1,trendyol,Sivas,NaN,181.820,18.180,200.00,Damla Beşer,trendyolekspress,Onaylandı
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92975,2036971893,001603,NaN,"Pikas Ahşap Özel İsim Baskılı, Kare Kedi Köpek...",2024-03-01 00:16:54,1,trendyol,Samsun,NaN,133.330,26.670,160.00,Gülizar Sarı,trendyolekspress,Onaylandı
92976,2036970386,002429,NaN,BTS3 Boyama Tuval Sayılı Set 25x35 Yunus,2024-03-01 00:16:07,1,trendyol,Çorum,NaN,154.170,30.830,185.00,zübeyde DEMİREL,trendyolekspress,Onaylandı
92977,0888556689,002369,NaN,Küçük Çelik Uçlu Kedi Köpek Evcil Hayvan Tırna...,2024-03-01 00:14:02,1,HepsiBurada,İzmir,NaN,33.325,6.665,39.99,İrem Erbey,hepsijet,Onaylandı
92978,2036956451,002284,yatak54896513739,Kalorifer Petek Radyatör Yatağı Kedi Geniş Ked...,2024-03-01 00:05:22,1,trendyol,Konya,NaN,142.500,28.500,171.00,Esma Söylemez,trendyolekspress,Onaylandı


In [21]:
df_entegra_select_var = df_entegra_select_var.drop_duplicates(subset="order_number", keep="first")
df_entegra_select_var = df_entegra_select_var.reset_index(drop=True)
df_entegra_select_var

,order_number,model,pov_productcode,product_name,datetime,total_product_quantity,entegration,invoice_city,invoice_country,total,tax,grand_total,company,cargo_company,status_name
0,2383110310,003362,NaN,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-09-05 10:54:27,1,trendyol,İstanbul,NaN,391.670,78.330,470.00,ali özdemir,trendyolekspress,Onaylandı
1,2383102583,004382,NaN,Çok Amaçlı Multi İnce Temizlik Fırçası Derz Ar...,2024-09-05 10:50:42,7,trendyol,İstanbul,NaN,395.530,48.470,444.00,Tuğba Sayar,trendyolekspress,Onaylandı
2,2383091113,001602,NaN,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-09-05 10:43:15,1,trendyol,Ankara,NaN,165.830,33.170,199.00,Tuba Koca,trendyolekspress,Onaylandı
3,2383085464,001602,NaN,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-09-05 10:40:26,2,trendyol,İstanbul,NaN,215.830,43.170,259.00,Doğa Özkan,trendyolekspress,Onaylandı
4,2383076067,002583,220647971025,"Portatif Katlanabilir Masa, Piknik Masası, Kam...",2024-09-05 10:34:55,1,trendyol,Sivas,NaN,181.820,18.180,200.00,Damla Beşer,trendyolekspress,Onaylandı
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92285,2036971893,001603,NaN,"Pikas Ahşap Özel İsim Baskılı, Kare Kedi Köpek...",2024-03-01 00:16:54,1,trendyol,Samsun,NaN,133.330,26.670,160.00,Gülizar Sarı,trendyolekspress,Onaylandı
92286,2036970386,002429,NaN,BTS3 Boyama Tuval Sayılı Set 25x35 Yunus,2024-03-01 00:16:07,1,trendyol,Çorum,NaN,154.170,30.830,185.00,zübeyde DEMİREL,trendyolekspress,Onaylandı
92287,0888556689,002369,NaN,Küçük Çelik Uçlu Kedi Köpek Evcil Hayvan Tırna...,2024-03-01 00:14:02,1,HepsiBurada,İzmir,NaN,33.325,6.665,39.99,İrem Erbey,hepsijet,Onaylandı
92288,2036956451,002284,yatak54896513739,Kalorifer Petek Radyatör Yatağı Kedi Geniş Ked...,2024-03-01 00:05:22,1,trendyol,Konya,NaN,142.500,28.500,171.00,Esma Söylemez,trendyolekspress,Onaylandı


In [22]:
df_entegra_select_var.loc[:,'datetime'] = pd.to_datetime(df_entegra_select_var.loc[:,'datetime'], format='%Y-%m-%d %H:%M:%S')
df_entegra_select_var.head(5)

,order_number,model,pov_productcode,product_name,datetime,total_product_quantity,entegration,invoice_city,invoice_country,total,tax,grand_total,company,cargo_company,status_name
0,2383110310,003362,NaN,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-09-05 10:54:27,1,trendyol,İstanbul,NaN,391.67,78.33,470.0,ali özdemir,trendyolekspress,Onaylandı
1,2383102583,004382,NaN,Çok Amaçlı Multi İnce Temizlik Fırçası Derz Ar...,2024-09-05 10:50:42,7,trendyol,İstanbul,NaN,395.53,48.47,444.0,Tuğba Sayar,trendyolekspress,Onaylandı
2,2383091113,001602,NaN,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-09-05 10:43:15,1,trendyol,Ankara,NaN,165.83,33.17,199.0,Tuba Koca,trendyolekspress,Onaylandı
3,2383085464,001602,NaN,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-09-05 10:40:26,2,trendyol,İstanbul,NaN,215.83,43.17,259.0,Doğa Özkan,trendyolekspress,Onaylandı
4,2383076067,002583,220647971025,"Portatif Katlanabilir Masa, Piknik Masası, Kam...",2024-09-05 10:34:55,1,trendyol,Sivas,NaN,181.82,18.18,200.0,Damla Beşer,trendyolekspress,Onaylandı


In [23]:
df_entegra_select_var.insert(3,"date",pd.to_datetime(df_entegra_select_var["datetime"]).dt.date)
df_entegra_select_var.insert(4,"time",pd.to_datetime(df_entegra_select_var["datetime"]).dt.time)
df_entegra_select_var.head(5)

,order_number,model,pov_productcode,date,time,product_name,datetime,total_product_quantity,entegration,invoice_city,invoice_country,total,tax,grand_total,company,cargo_company,status_name
0,2383110310,003362,NaN,2024-09-05,10:54:27,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-09-05 10:54:27,1,trendyol,İstanbul,NaN,391.67,78.33,470.0,ali özdemir,trendyolekspress,Onaylandı
1,2383102583,004382,NaN,2024-09-05,10:50:42,Çok Amaçlı Multi İnce Temizlik Fırçası Derz Ar...,2024-09-05 10:50:42,7,trendyol,İstanbul,NaN,395.53,48.47,444.0,Tuğba Sayar,trendyolekspress,Onaylandı
2,2383091113,001602,NaN,2024-09-05,10:43:15,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-09-05 10:43:15,1,trendyol,Ankara,NaN,165.83,33.17,199.0,Tuba Koca,trendyolekspress,Onaylandı
3,2383085464,001602,NaN,2024-09-05,10:40:26,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-09-05 10:40:26,2,trendyol,İstanbul,NaN,215.83,43.17,259.0,Doğa Özkan,trendyolekspress,Onaylandı
4,2383076067,002583,220647971025,2024-09-05,10:34:55,"Portatif Katlanabilir Masa, Piknik Masası, Kam...",2024-09-05 10:34:55,1,trendyol,Sivas,NaN,181.82,18.18,200.0,Damla Beşer,trendyolekspress,Onaylandı


In [24]:
object_columns = df_entegra_select_var.select_dtypes(include='object')
object_columns

,order_number,model,pov_productcode,date,time,product_name,entegration,invoice_city,invoice_country,company,cargo_company,status_name
0,2383110310,003362,NaN,2024-09-05,10:54:27,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",trendyol,İstanbul,NaN,ali özdemir,trendyolekspress,Onaylandı
1,2383102583,004382,NaN,2024-09-05,10:50:42,Çok Amaçlı Multi İnce Temizlik Fırçası Derz Ar...,trendyol,İstanbul,NaN,Tuğba Sayar,trendyolekspress,Onaylandı
2,2383091113,001602,NaN,2024-09-05,10:43:15,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",trendyol,Ankara,NaN,Tuba Koca,trendyolekspress,Onaylandı
3,2383085464,001602,NaN,2024-09-05,10:40:26,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",trendyol,İstanbul,NaN,Doğa Özkan,trendyolekspress,Onaylandı
4,2383076067,002583,220647971025,2024-09-05,10:34:55,"Portatif Katlanabilir Masa, Piknik Masası, Kam...",trendyol,Sivas,NaN,Damla Beşer,trendyolekspress,Onaylandı
...,...,...,...,...,...,...,...,...,...,...,...,...
92285,2036971893,001603,NaN,2024-03-01,00:16:54,"Pikas Ahşap Özel İsim Baskılı, Kare Kedi Köpek...",trendyol,Samsun,NaN,Gülizar Sarı,trendyolekspress,Onaylandı
92286,2036970386,002429,NaN,2024-03-01,00:16:07,BTS3 Boyama Tuval Sayılı Set 25x35 Yunus,trendyol,Çorum,NaN,zübeyde DEMİREL,trendyolekspress,Onaylandı
92287,0888556689,002369,NaN,2024-03-01,00:14:02,Küçük Çelik Uçlu Kedi Köpek Evcil Hayvan Tırna...,HepsiBurada,İzmir,NaN,İrem Erbey,hepsijet,Onaylandı
92288,2036956451,002284,yatak54896513739,2024-03-01,00:05:22,Kalorifer Petek Radyatör Yatağı Kedi Geniş Ked...,trendyol,Konya,NaN,Esma Söylemez,trendyolekspress,Onaylandı


In [25]:
for column in object_columns.columns:
    df_entegra_select_var.loc[:,f"{column}"] = df_entegra_select_var.loc[:,f"{column}"].apply(lambda x: convert_one_character_letter(x))
    df_entegra_select_var.loc[:,f"{column}"] = df_entegra_select_var.loc[:,f"{column}"].apply(lambda x: whitespace_del(x))
    df_entegra_select_var.loc[:,f"{column}"] = df_entegra_select_var.loc[:,f"{column}"].apply(lambda x: title_func(x))
    
df_entegra_select_var.head(10)

,order_number,model,pov_productcode,date,time,product_name,datetime,total_product_quantity,entegration,invoice_city,invoice_country,total,tax,grand_total,company,cargo_company,status_name
0,2383110310,003362,Nan,2024-09-05,10:54:27,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-09-05 10:54:27,1,Trendyol,İstanbul,Nan,391.6700,78.3300,470.00,Ali Özdemir,Trendyolekspress,Onaylandı
1,2383102583,004382,Nan,2024-09-05,10:50:42,Çok Amaçlı Multi İnce Temizlik Fırçası Derz Ar...,2024-09-05 10:50:42,7,Trendyol,İstanbul,Nan,395.5300,48.4700,444.00,Tuğba Sayar,Trendyolekspress,Onaylandı
2,2383091113,001602,Nan,2024-09-05,10:43:15,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-09-05 10:43:15,1,Trendyol,Ankara,Nan,165.8300,33.1700,199.00,Tuba Koca,Trendyolekspress,Onaylandı
3,2383085464,001602,Nan,2024-09-05,10:40:26,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-09-05 10:40:26,2,Trendyol,İstanbul,Nan,215.8300,43.1700,259.00,Doğa Özkan,Trendyolekspress,Onaylandı
4,2383076067,002583,220647971025,2024-09-05,10:34:55,"Portatif Katlanabilir Masa, Piknik Masası, Kam...",2024-09-05 10:34:55,1,Trendyol,Sivas,Nan,181.8200,18.1800,200.00,Damla Beşer,Trendyolekspress,Onaylandı
5,2383074871,001602,Nan,2024-09-05,10:34:41,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-09-05 10:34:41,1,Trendyol,Denizli,Nan,165.8300,33.1700,199.00,Sare Hacer Var,Trendyolekspress,Onaylandı
6,2383069120,002643,290894121988,2024-09-05,10:30:18,Kaymaz Taban Tekli Post Peluş Banyo Paspası,2024-09-05 10:30:18,1,Trendyol,Bursa,Nan,90.0000,9.0000,99.00,Safinaz Dinç,Trendyolekspress,Onaylandı
7,2383066275,002263,Ayk8683372407015Çkmk,2024-09-05,10:29:26,"İksa Devrilmez Ayaklı Portatif Kamp Ocağı, Pik...",2024-09-05 10:29:26,1,Trendyol,Ankara,Nan,304.1700,60.8300,365.00,Şeyma Ödemi̇ş,Mng,Onaylandı
8,4575428431,003548,Nan,2024-09-05,10:28:49,"Yüzük Zikirmatik Karışık Renk, Yeni Nesil Ziki...",2024-09-05 10:28:49,2,Hepsiburada,Samsun,Nan,49.9833,9.9967,59.98,Zeynep Rana Karaağaç,Hepsijet,Onaylandı
9,2383064808,004159,Nan,2024-09-05,10:28:11,Spider Örümcek Oyun Çadırı Çift Yönlü Giriş Çı...,2024-09-05 10:28:11,1,Trendyol,Mersin,Nan,329.1700,65.8300,395.00,Senem Cekli,Trendyolekspress,Onaylandı


##### Entegra Products

In [26]:
df_product_master = pd.read_excel(r"C:\Users\user\Desktop\Data Analysis\Adoba\Sales_Report\Data\Entegra\Product Name\Master\Entegra Model Product Name Update (19.09.2024).xlsx")
df_product_master

,model,product_name
0,003362,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö..."
1,001602,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ..."
2,003954,Awox Barmix Çubuk Blender 1000 W
3,004286,Awox Orbit Gizli Rezistanslı Su Isıtıcı Kettle...
4,002656,Sprey Başlıklı Amber Cam 10 Ml Boş - Yedek Şişe
...,...,...
1258,003249,Merinos Relax Çift Kişilik Battaniye Color Li...
1259,004303,Procli̇s Deve Dikeni Yağı %100 Doğal Bitkisel ...
1260,003060,Merinos Eternity Fitilli Peluş Çift Kişilik Ne...
1261,003593,"Zirkon Taşlı Çelik Dünya Gezegen Gold Kolye, K..."


In [27]:
# df_entegra_select_product = df_entegra_select_var[["model","pov_productcode","product_name"]]
df_entegra_select_product = df_entegra_select_var[["model","product_name"]]
df_entegra_select_product

,model,product_name
0,003362,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö..."
1,004382,Çok Amaçlı Multi İnce Temizlik Fırçası Derz Ar...
2,001602,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ..."
3,001602,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ..."
4,002583,"Portatif Katlanabilir Masa, Piknik Masası, Kam..."
...,...,...
92285,001603,"Pikas Ahşap Özel İsim Baskılı, Kare Kedi Köpek..."
92286,002429,Bts3 Boyama Tuval Sayılı Set 25X35 Yunus
92287,002369,Küçük Çelik Uçlu Kedi Köpek Evcil Hayvan Tırna...
92288,002284,Kalorifer Petek Radyatör Yatağı Kedi Geniş Ked...


In [28]:
df_entegra_select_product["model"].nunique()

975

In [29]:
df_entegra_select_product["product_name"].nunique()

993

In [30]:
df_entegra_select_product[df_entegra_select_product.duplicated()]

,model,product_name
3,001602,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ..."
5,001602,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ..."
10,002583,"Portatif Katlanabilir Masa, Piknik Masası, Kam..."
11,003548,"Yüzük Zikirmatik Karışık Renk, Yeni Nesil Ziki..."
12,001602,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ..."
...,...,...
92285,001603,"Pikas Ahşap Özel İsim Baskılı, Kare Kedi Köpek..."
92286,002429,Bts3 Boyama Tuval Sayılı Set 25X35 Yunus
92287,002369,Küçük Çelik Uçlu Kedi Köpek Evcil Hayvan Tırna...
92288,002284,Kalorifer Petek Radyatör Yatağı Kedi Geniş Ked...


In [31]:
df_entegra_select_product = df_entegra_select_product.drop_duplicates()
df_entegra_select_product

,model,product_name
0,003362,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö..."
1,004382,Çok Amaçlı Multi İnce Temizlik Fırçası Derz Ar...
2,001602,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ..."
4,002583,"Portatif Katlanabilir Masa, Piknik Masası, Kam..."
6,002643,Kaymaz Taban Tekli Post Peluş Banyo Paspası
...,...,...
91758,001656,Guard Mat Siyah Klasik Deri Erkek Cüzdanı 171
91791,003412,"Levi Çift Kişilik Keten Yatak Örtüsü, Çift Kiş..."
91838,004034,Kadın Erkek Desenli 4034
91903,002586,Bts20 Boyama Tuval Sayılı Set 25X35


In [32]:
master_model = set(df_product_master["model"])
new_model = set(df_entegra_select_product["model"])

In [33]:
new_model_list = list(new_model.difference(master_model))

In [34]:
df_entegra_select_new_product = df_entegra_select_product[df_entegra_select_product["model"].isin(new_model_list)]
df_entegra_select_new_product.reset_index(drop=True, inplace=True)
df_entegra_select_new_product

,model,product_name


In [35]:
# df_entegra_select_new_product.to_excel(fr"{output_path}\Entegra Model New Product Name ({variable_date}).xlsx", index=False)  # Adoba..Data Folder

#### Location

In [36]:
def reverse_geocoding(lat, lon):
    '''This function is used for giving city or country name from longitude latitude value \n
    reverse_geocoding(lat, lon): lat is geographical lattitude and lon is geographical longitude \n
    ex. \n
    df['address'] = np.vectorize(reverse_geocoding)(df['Latitude'], df['Longitude'])'''
    try:
        location = geolocator.reverse(Point(lat, lon))
        return location.raw['display_name']
    except:
        return None

In [37]:
def findGeocode(city):
    '''This function is used for giving latitude and longitude values from city or country name \n
    findGeocode(city): city is a city or country \n
    ex. \n
    loc = findGeocode(i) \n 
    lat = loc.latitude \n
    lon = loc.longitude''' 
       
    # try and catch is used to overcome 
    # the exception thrown by geolocator 
    # using geocodertimedout   
    try: 
          
        # Specify the user_agent as your 
        # app name it should not be none 
        geolocator = Nominatim(user_agent="tesst") 
          
        return geolocator.geocode(city) 
      
    except GeocoderTimedOut: 
          
        return findGeocode(city)     

In [38]:
df_city_location_master = pd.read_excel(r"C:\Users\user\Desktop\Data Analysis\Adoba\Sales_Report\Data\Geo Location\Master\Entegra City Location.xlsx")
df_city_location_master.drop("id", axis=1, inplace=True)
df_city_location_master

,city,country,invoice_city,latitude,longitude
0,Adana,Turkey,Adana,37.143800,35.498409
1,Adiyaman,Turkey,Adıyaman,37.763750,38.276348
2,Adiyaman,Turkey,Adiyaman,37.763750,38.276348
3,Afyonkarahisar,Turkey,Afyonkarahisar,38.685273,30.642741
4,Afyonkarahisar,Turkey,Afyon,38.685273,30.642741
...,...,...,...,...,...
262,Yanbu,Saudi Arabia,Yanbu,24.094893,38.070525
263,Az Zulfi,Saudi Arabia,Zulfi,26.311619,44.832951
264,Qairawan,Tunisia,Qairawan,35.673549,10.098636
265,Al Khodh,Oman,Al Khodh,23.565997,58.118169


In [39]:
df_entegra_select_var["invoice_city"].nunique()

267

In [40]:
df_entegra_select_var[df_entegra_select_var["invoice_city"] == "Nan"]

,order_number,model,pov_productcode,date,time,product_name,datetime,total_product_quantity,entegration,invoice_city,invoice_country,total,tax,grand_total,company,cargo_company,status_name
42703,48014,Nan,Nan,2024-06-03,22:00:13,Nan,2024-06-03 22:00:13,0,Woocommerce,Nan,Nan,0.0,0.0,0.0,Nan,Diger,Yeni Siparis


In [41]:
nan_index = df_entegra_select_var[df_entegra_select_var["invoice_city"] == "Nan"].index[0]
nan_index

42703

In [42]:
new_city = set(df_entegra_select_var["invoice_city"])
master_city = set(df_city_location_master["invoice_city"])
new_city_list = list(new_city.difference(master_city))
# new_city_list

In [43]:
df_entegra_city_select = df_entegra_select_var[df_entegra_select_var["invoice_city"].isin(new_city_list)]
df_entegra_city_select = df_entegra_city_select[["invoice_city", "invoice_country"]]
df_entegra_city_select = df_entegra_city_select.drop_duplicates(subset="invoice_city")
df_entegra_city_select = df_entegra_city_select.reset_index(drop=True)
df_entegra_city_select

,invoice_city,invoice_country
0,Dumbrăvița,Romanya
1,Bratislava,Slovakya
2,Sabirabad,Azerbaycan
3,Cluj-Napoca,Romanya
4,Ramnicu Valcea,Romanya
5,Oyoon Al Jawa,Suudi Arabistan
6,Cadca,Slovakya
7,Anak,Suudi Arabistan
8,Zalau,Romanya
9,Qurayat,Suudi Arabistan


In [44]:
# df_entegra_city_select.to_excel(fr"{output_path}\Entegra New City ({variable_date}).xlsx", index=False)

In [45]:
df_entegra_new_city_select = pd.read_excel(fr"{output_path}\Entegra New City Edit (01.04.2024-05.09.2024).xlsx")
df_entegra_new_city_select

,invoice_city,invoice_country
0,Dumbrăvița,Romanya
1,Bratislava,Slovakya
2,Sabirabad,Azerbaycan
3,Cluj-Napoca,Romanya
4,Ramnicu Valcea,Romanya
5,Oyoon Al Jawa,Suudi Arabistan
6,Cadca,Slovakya
7,Anak,Suudi Arabistan
8,Zalau,Romanya
9,Qurayat,Suudi Arabistan


In [46]:
new_city_list = list(df_entegra_new_city_select["invoice_city"])

In [47]:
geolocator = Nominatim(user_agent="test")

# each value from city column 
# will be fetched and sent to 
# function find_geocode  
city = [] 
latitude = [] 
longitude = []

# for i in (df_entegra_city_select["invoice_city"]):
for i in new_city_list: 
      
    if findGeocode(i) != None: 
           
        loc = findGeocode(i) 
          
        # coordinates returned from  
        # function is stored into 
        # two separate list 
        city.append(i)
        latitude.append(loc.latitude) 
        longitude.append(loc.longitude) 
       
    # if coordinate for a city not 
    # found, insert "NaN" indicating  
    # missing value  
    else:
        city.append(np.nan) 
        latitude.append(np.nan) 
        longitude.append(np.nan)

In [48]:
df_entegra_new_city_select["latitude"] = latitude
df_entegra_new_city_select["longitude"] = longitude 
df_entegra_new_city_select

,invoice_city,invoice_country,latitude,longitude
0,Dumbrăvița,Romanya,45.801876,21.247858
1,Bratislava,Slovakya,48.159260,17.139659
2,Sabirabad,Azerbaycan,39.889001,48.795703
3,Cluj-Napoca,Romanya,46.769379,23.589954
4,Ramnicu Valcea,Romanya,45.103173,24.364721
5,Oyoon Al Jawa,Suudi Arabistan,NaN,NaN
6,Cadca,Slovakya,49.437515,18.789604
7,Anak,Suudi Arabistan,16.223299,121.657201
8,Zalau,Romanya,10.392447,9.012325
9,Qurayat,Suudi Arabistan,23.261044,58.920269


In [49]:
# df_entegra_new_city_select.to_excel(fr"{output_path}\Entegra New City Location ({variable_date}).xlsx", index=False)  # Adoba..Data Folder